In [1]:
import sys
#print(sys.path)
sys.path.append("/Users/raroito/PycharmProjects/facial_expression_recognition/src/")

from utils import ModelImporter, Preprocessing
import torch
import pandas as pd
import numpy as np

# Analize Test Data Results

In [2]:
pre = Preprocessing('fer2013')
pre.load_data(filename='test_public_norm.csv', name='test')

X_df = pre.get(name='test').drop(columns=['emotion'])
y_df = pre.get(name='test')['emotion']

In [3]:
dtype = torch.float
device = torch.device("cpu")

model_name = 'cnn_simple'

m_importer = ModelImporter('fer2013_reduced')

n_classes = 7
n_epochs = 100
model = m_importer.load_nn_model(model_name, 0, n_classes, n_epochs)

load model CnnSimple(
  (conv1): Conv2d(1, 15, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=8640, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=7, bias=True)
)


In [4]:
X_test = model.reshape_data(torch.tensor(X_df.values, device=device, dtype=dtype))
y_test = torch.tensor(y_df.values, device=device, dtype=torch.long)

In [5]:
y_pred_prob = model(X_test)

In [6]:
y_pred = y_pred_prob.argmax(1)

In [7]:
y_pred_prob

tensor([[8.6183e-01, 3.3330e-13, 8.7976e-01,  ..., 3.8531e-04, 6.6889e-01,
         5.0816e-03],
        [5.9335e-01, 3.2782e-14, 2.7200e-02,  ..., 9.3483e-01, 2.0083e-02,
         3.7060e-01],
        [2.9857e-01, 1.4620e-11, 6.1640e-01,  ..., 7.7929e-01, 3.2110e-02,
         9.6057e-01],
        ...,
        [4.2728e-01, 2.7254e-16, 8.4799e-01,  ..., 9.9984e-01, 1.6063e-03,
         9.9993e-01],
        [9.6959e-01, 7.1289e-12, 6.6486e-02,  ..., 9.5458e-01, 5.9755e-05,
         9.5886e-01],
        [8.5300e-01, 7.3422e-11, 2.0647e-02,  ..., 9.7898e-01, 2.9230e-04,
         9.4654e-01]], grad_fn=<SigmoidBackward>)

In [8]:
y_pred

tensor([2, 4, 6,  ..., 6, 0, 4])

In [9]:
results = pd.DataFrame()

In [10]:
results['y_test'] = y_test
results['y_pred'] = y_pred
results['right'] = (y_pred == y_test).float()
results['ones'] = 1

In [11]:
results.tail()

,y_test,y_pred,right,ones
3584,4,0,0.0,1
3585,3,3,1.0,1
3586,4,6,0.0,1
3587,4,0,0.0,1
3588,4,4,1.0,1


In [12]:
report = results.groupby('y_test').sum()
report

,y_pred,right,ones
y_test,,,
0,1591,69.0,467
1,186,0.0,56
2,1809,79.0,496
3,3035,600.0,895
4,2413,149.0,653
5,1734,197.0,415
6,2652,288.0,607


In [13]:
report['%'] = report['right']/report['ones']
report

,y_pred,right,ones,%
y_test,,,,
0,1591,69.0,467,0.147752
1,186,0.0,56,0.000000
2,1809,79.0,496,0.159274
3,3035,600.0,895,0.670391
4,2413,149.0,653,0.228178
5,1734,197.0,415,0.474699
6,2652,288.0,607,0.474465


In [ ]:
#class 3 is somhow well recognized (also is the class with more samples) 
#whereas not a single sample of class 2 was properly detected

# Analize the Training Dataset

In [30]:
pre = Preprocessing('fer2013')
pre.load_data(filename='train_norm.csv', name='train')

X_df = pre.get(name='train').drop(columns=['emotion'])
y_df = pre.get(name='train')['emotion']

In [32]:
X_train= model.reshape_data(torch.tensor(X_df.values, device=device, dtype=dtype))
y_train = torch.tensor(y_df.values, device=device, dtype=torch.long)

In [33]:
y_pred_prob = model(X_train)

In [34]:
y_pred = y_pred_prob.argmax(1)

In [35]:
y_pred_prob

tensor([[2.8741e-01, 2.9205e-11, 5.1514e-01,  ..., 9.3239e-01, 1.5750e-03,
         4.3642e-01],
        [8.8649e-01, 5.4478e-12, 7.6732e-01,  ..., 9.8481e-01, 1.7405e-03,
         3.1425e-01],
        [3.4288e-02, 2.0129e-13, 1.0720e-01,  ..., 7.5036e-02, 1.3333e-01,
         3.3199e-01],
        ...,
        [6.5698e-01, 1.8350e-12, 9.4406e-01,  ..., 9.3196e-01, 2.0606e-02,
         1.3956e-01],
        [4.7786e-02, 1.2951e-11, 3.6627e-01,  ..., 2.8148e-01, 7.6672e-01,
         4.0782e-01],
        [9.0945e-02, 2.1948e-13, 8.2959e-01,  ..., 3.8975e-01, 4.6363e-01,
         2.9803e-01]], grad_fn=<SigmoidBackward>)

In [38]:
y_pred

tensor([4, 4, 3,  ..., 2, 5, 2])

In [39]:
y_train

tensor([0, 0, 2,  ..., 4, 0, 4])

In [42]:
results_train['y_train'] = y_train
results_train['y_pred'] = y_pred
results_train['right'] = (y_pred == y_train).float()
results_train['ones'] = 1

In [43]:
report_train = results_train.groupby('y_train').sum()
report_train

,y_pred,right,ones
y_train,,,
0,13927,235.0,3995
1,1523,0.0,436
2,14569,569.0,4097
3,23724,5378.0,7215
4,18187,2040.0,4830
5,12650,1252.0,3171
6,20387,1509.0,4965


In [44]:
report_train['%'] = report_train['right']/report_train['ones']
report_train

,y_pred,right,ones,%
y_train,,,,
0,13927,235.0,3995,0.058824
1,1523,0.0,436,0.000000
2,14569,569.0,4097,0.138882
3,23724,5378.0,7215,0.745392
4,18187,2040.0,4830,0.422360
5,12650,1252.0,3171,0.394828
6,20387,1509.0,4965,0.303927


In [ ]:
#similar results with the train data as with the 